#### INSTALL AND IMPORT DEPENDENCIES

In [17]:
import cv2
import numpy as np
import pandas as pd
import os 
import matplotlib.pyplot as plt
import time
import mediapipe as mp
from pathlib import Path
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

#### KEYPOINTS WITH MP HOLISTIC


In [3]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
mp_drawing_style = mp.solutions.drawing_styles

In [4]:
def mediapipe_detection(image,model):
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    return image,results


In [5]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret,frame = cap.read()
        if not ret:
            break

        frame,results = mediapipe_detection(frame,holistic)

        cv2.imshow('frame',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

#### GET SEQUENCE AND LABELS FOR MODELS

In [6]:
actions = ['noshot','shot']
label_map = {label:num for num,label in enumerate(actions)}
label_map

{'noshot': 0, 'shot': 1}

In [9]:
sequences,labels = [],[]
for action in actions:
    print(action)
    action_path = Path('shot_detection_images',action)
    action_files = [x for x in action_path.glob('*')]
    for player_file in action_files:
        window = []
        if not player_file.is_dir():
            continue
        for i in range(25):
            image_array = np.load(str(Path(player_file,f"{i}.npy")))
            window.append(image_array)
        sequences.append(window)
        labels.append(label_map[action])
        

noshot
shot


In [16]:
x = np.array(sequences)
y = to_categorical(labels).astype(int)

In [65]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

#### BUILD AND TRAIN LSTM NEURAL NETWORK

In [66]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense
from tensorflow.keras.callbacks import TensorBoard

In [25]:
log_dir = os.path.join('logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [67]:
model_lstm = Sequential()
model_lstm.add(LSTM(64,return_sequences=True,activation='relu',input_shape=(25,1662)))
model_lstm.add(LSTM(128,return_sequences=True,activation='relu'))
model_lstm.add(LSTM(64,return_sequences=False,activation='relu'))
model_lstm.add(Dense(64,activation='relu'))
model_lstm.add(Dense(32,activation='relu'))
model_lstm.add(Dense(2,activation='softmax'))
model_lstm.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_15 (LSTM)              (None, 25, 64)            442112    
                                                                 
 lstm_16 (LSTM)              (None, 25, 128)           98816     
                                                                 
 lstm_17 (LSTM)              (None, 64)                49408     
                                                                 
 dense_14 (Dense)            (None, 64)                4160      
                                                                 
 dense_15 (Dense)            (None, 32)                2080      
                                                                 
 dense_16 (Dense)            (None, 2)                 66        
                                                                 
Total params: 596,642
Trainable params: 596,642
Non-tr

In [68]:
model_lstm.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [70]:
model_lstm.fit(x_train,y_train,epochs=150,callbacks=[tb_callback])

Epoch 1/150
1/1 [==============================] - 0s 299ms/step - loss: 4.9714 - accuracy: 0.8182
Epoch 2/150
1/1 [==============================] - 0s 90ms/step - loss: 3.0235 - accuracy: 0.8182
Epoch 3/150
1/1 [==============================] - 0s 87ms/step - loss: 3.9717 - accuracy: 0.8182
Epoch 4/150
1/1 [==============================] - 0s 88ms/step - loss: 5.0189 - accuracy: 0.8182
Epoch 5/150
1/1 [==============================] - 0s 103ms/step - loss: 9.0743 - accuracy: 0.6818
Epoch 6/150
1/1 [==============================] - 0s 119ms/step - loss: 1.5969 - accuracy: 0.9091
Epoch 7/150
1/1 [==============================] - 0s 133ms/step - loss: 0.9221 - accuracy: 0.9545
Epoch 8/150
1/1 [==============================] - 0s 123ms/step - loss: 2.5179 - accuracy: 0.9091
Epoch 9/150
1/1 [==============================] - 0s 118ms/step - loss: 4.6720 - accuracy: 0.9091
Epoch 10/150
1/1 [==============================] - 0s 140ms/step - loss: 7.3137 - accuracy: 0.8636
Epoch 11/150

In [71]:
x_test.shape

(6, 25, 1662)

In [72]:
y_pred = model_lstm.predict(x_test)

1/1 [==============================] - 0s 479ms/step


In [73]:
np.argmax(y_pred,axis=1)

array([1, 1, 0, 1, 1, 0], dtype=int64)

In [74]:
y_test

array([[1, 0],
       [0, 1],
       [1, 0],
       [0, 1],
       [1, 0],
       [1, 0]])

In [76]:
from sklearn.metrics import confusion_matrix,accuracy_score

In [63]:
np.argmax(y_pred,axis=1)

array([1, 1, 0], dtype=int64)

In [78]:
yhat = model.predict(x_train)


1/1 [==============================] - 0s 187ms/step


In [82]:
yhat = np.argmax(yhat,axis=1).tolist()

In [79]:
y_train = np.argmax(y_train,axis=1).tolist()

In [83]:
accuracy_score(y_train,yhat)

1.0

In [85]:
model_lstm.save('shot_detection.h5')

In [86]:
del model_lstm

In [87]:
model.load('shot_detection.h5')

AttributeError: 'Sequential' object has no attribute 'load'